In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/07/17 19:03:44 WARN Utils: Your hostname, spark-master, resolves to a loopback address: 127.0.1.1; using 10.168.136.115 instead (on interface ens3)
25/07/17 19:03:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/17 19:03:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/07/17 19:03:44 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


### Read option -1

In [6]:
people_df = spark.read.load(
    "file:///data/Drivers.csv",
    format="csv",
    sep=",",
    inferSchema="true",
    header="true"
)

In [9]:
people_df.show(5)

+-------------------+--------------------+--------------------+--------------+---------------+
|DriverLicenseNumber|                Name|                Type|ExpirationDate|LastDateUpdated|
+-------------------+--------------------+--------------------+--------------+---------------+
|            5430898|   ABDEL-BAR,ESLAM,M|MEDALLION TAXI DR...|    04/12/2023|     04/22/2020|
|            5363749|ABDOUSAMADOV,ALIC...|MEDALLION TAXI DR...|    06/01/2020|     04/22/2020|
|            5534446|  ABDUHALIKOV,RUSTAM|MEDALLION TAXI DR...|    06/16/2020|     04/22/2020|
|            5935702|   ABDULLAEV,JONIBEK|MEDALLION TAXI DR...|    03/14/2022|     04/22/2020|
|            5255097|ABDULNABI,MASHHOUR,H|MEDALLION TAXI DR...|    03/16/2021|     04/22/2020|
+-------------------+--------------------+--------------------+--------------+---------------+
only showing top 5 rows


In [11]:
people_df.select('Name').write.save("file:///data/test.parquet")

In [12]:
people_df.rdd.getNumPartitions()

3

### Read option 2

In [22]:
df = spark.sql("SELECT * FROM parquet.`file:///data/test.parquet`")
df.count()

154626

### Save modes

In [16]:
# Default - ErrorIfExists
people_df.select('Name').write.save("file:///data/test.parquet")

AnalysisException: [PATH_ALREADY_EXISTS] Path file:/data/test.parquet already exists. Set mode as "overwrite" to overwrite the existing path. SQLSTATE: 42K04

In [26]:
# Ignore
people_df.select('Name').write.save("file:///data/test.parquet",mode="ignore")

In [18]:
# overwrite

people_df.select('Name').write.save("file:///data/test.parquet",mode="overwrite")

In [23]:
df = spark.sql("SELECT * FROM parquet.`file:///data/test.parquet`")
df.count()

154626

In [24]:
# Append

people_df.select('Name').write.save("file:///data/test.parquet",mode="append")

In [25]:
df = spark.sql("SELECT * FROM parquet.`file:///data/test.parquet`")
df.count()

309252

In [27]:
spark.stop()